First, a note: Here, we'll be using import statements as if you were in the pandasGEXpress directory (eg, from wherever you've forked l1ktools: l1ktools/python/broadinstitute_cmap/io/pandasGEXpress). If you'd like to use nested imports, first run setup.py (found in l1ktools/python/broadinstitute_cmap; type 

```
$ python setup.py --help 
```

on the command line for details. Then, from your python session, you should be able to access pandasGEXpress with import statements in the following form: 

```python
from broadinstitute_cmap.io.pandasGEXpress import [package]
```


#### Table of Contents

# Reading in gct/x files

## If you'd like to read in an entire (.gct or .gctx) file to a GCToo instance:

In [1]:
from broadinstitute_cmap.io.pandasGEXpress import parse
my_gctoo = parse.parse("functional_tests/both_metadata_example_n1476x978.gctx")
my_gctoo

## If you're using the .gctx format, you can also only read in row or column metadata from a .gctx file. 

In [2]:
# read in row metadata only 
row_metadata_only = parse.parse("functional_tests/mini_gctx_with_metadata_n2x3.gctx", meta_only="row")
row_metadata_only

In [3]:
# read in column metadata only
col_metadata_only = parse.parse("functional_tests/mini_gctx_with_metadata_n2x3.gctx", meta_only="col")
col_metadata_only

## You can also read in only a certain subset of rids and/or cids to a GCToo instance.

Practically speaking, this is more useful for GCTX files than GCT files, since (as a text file) you'll need to read in the entire GCT file anyway. **You'll need to have a list of desired rids and/or cids already (can be obtained from reading only metadata in first, then subsetting)**


In [4]:
my_rids = ["218597_s_at", "200814_at"]
my_cids = ["LJP005_A375_24H:BRD-K76908866:10"]

# you can subset by rids, cids, or both rids and cids 
mini_gctoo_subset = parse.parse("functional_tests/mini_gctx_with_metadata_n2x3.gctx", rid = my_rids, cid= my_cids)
mini_gctoo_subset.row_metadata_df

rhd,pr_analyte_id,pr_analyte_num,pr_gene_id,pr_model_id
rid,,,,
200814_at,Analyte 11,11.0,5720.0,NaN
218597_s_at,Analyte 12,12.0,55847.0,NaN


In [5]:
mini_gctoo_subset.data_df.shape

(2, 1)

In [6]:
mini_gctoo_subset.col_metadata_df

chd,bead_batch,bead_revision
cid,,
LJP005_A375_24H:BRD-K76908866:10,b19,r2


# Construct a GCToo object programatically

For instance, say we want to create a GCToo instance of a file we've read in (for this example, let's use `my_gctoo`) that lacks metadata.

In [8]:
import pandas as pd
from broadinstitute_cmap.io.pandasGEXpress import GCToo
from broadinstitute_cmap.io.pandasGEXpress import parse

my_gctoo = parse.parse("functional_tests/both_metadata_example_n1476x978.gctx")

minimal_row_meta = pd.DataFrame(index = my_gctoo.row_metadata_df.index)
minimal_col_meta = pd.DataFrame(index = my_gctoo.col_metadata_df.index)

data_only_gctoo = GCToo.GCToo(data_df = my_gctoo.data_df, 
 	row_metadata_df = minimal_row_meta, col_metadata_df = minimal_col_meta)

data_only_gctoo.row_metadata_df.shape

(978, 0)

In [9]:
data_only_gctoo.col_metadata_df.shape

(1476, 0)

In [10]:
data_only_gctoo.data_df.shape == my_gctoo.data_df.shape

True

# Merging gct/x files


## Several files (with consistent name prefix), from the command line

Say you have a bunch of files that start with 'LINCS_GCP' in your Downloads folder that you want to concatenate **horizontally**. Type the following in your command line:

```
python /Users/some_name/code/l1ktools/python/broadinstitute_cmap/io/GCToo/concat_gctoo.py --file_wildcard '/Users/some_name/Downloads/LINCS_GCP*' --concat_direction horz
```

This will save a file called `concated.gctx` in your current directory. Make sure that the wildcard is in quotes!

## Two files to concatenate, from a Python session 

You have 2 GCToo objects in memory that you want to concatenate. hstack and vstack are the methods in concat_gctoo.py that actually do the concatenation. From within the Python console or script where you have your 2 GCToos (gct1 & gct2), type the following:

In [14]:
from broadinstitute_cmap.io.pandasGEXpress import concat_gctoo
from broadinstitute_cmap.io.pandasGEXpress import parse

gct1 = parse.parse("functional_tests/test_merge_left.gct")
gct2 = parse.parse("functional_tests/test_merge_right.gct")
merged_gcts = concat_gctoo.hstack([gct1, gct2])

gct1.data_df.shape

(4, 3)

In [15]:
gct2.data_df.shape

(3, 3)

In [16]:
merged_gcts.data_df.shape

(4, 6)

Note that there are also 2 optional arguments to hstack and vstack to work around certain obstacles common to concatenation. The full list of arguments to hstack and vstack are:

```
hstack(gctoos, fields_to_remove=[], reset_ids=False)
vstack(gctoos, fields_to_remove=[], reset_ids=False)

```

1) fields_to_remove: If the 'common' metadata contains fields that are not the same in all files, then you will need to remove these fields using the fields_to_remove argument.

2) reset_ids: If the 'concatenated' metadata ids are not unique between different files, and you try to concatenate the files, an invalid GCToo would be formed (duplicate ids). To overcome this, use the reset_sample_ids argument. This will move the 'new' metadata ids to a new metadata field and replace the original ids with unique integers.

## Two files to concatenate, from the command line 

Say you have 2 files that you want to concatenate **vertically**: /Users/some_name/file_to_concatenate1.gct and /Users/some_name/file_to_concatenate2.gct. Type the following in your command line:

```
python /Users/some_name/code/l1ktools/python/broadinstitute_cmap/io/GCToo/concat_gctoo.py --list_of_gct_paths /Users/some_name/file_to_concatenate1.gct /Users/some_name/file_to_concatenate2.gct --concat_direction vert
```

Optional arguments are as above (for hstack and vstack). 

# Slicing gct/x files 

Can be done from the command line or an active python session (assume you've already parsed in a file and have an instance of GCToo called mini_gctoo). For this use case, you'll need to have a list of desired and/or undesired rids and/or cids already (can be obtained from doing Use Case 2 or by other means). For example:

In [17]:
from broadinstitute_cmap.io.pandasGEXpress import parse
from broadinstitute_cmap.io.pandasGEXpress import slice_gct

mini_gctoo = parse.parse("functional_tests/mini_gctx_with_metadata_n2x3.gctx")

interesting_rids = ["218597_s_at", "200814_at"]
interesting_cids = ["LJP005_A375_24H:BRD-K76908866:10"]
boring_rids = ["217140_s_at"]

sliced_gctoo = slice_gct.slice_gctoo(mini_gctoo, rid = interesting_rids, cid = interesting_cids, exclude_rid = boring_rids)

sliced_gctoo.data_df.shape

(2, 1)

# Add/delete annotations

First, let's take the simplest case. You have a GCToo instance with metadata and want to add a list **that you know is ordered already like the rids of the row metadata**. 

In [18]:
from broadinstitute_cmap.io.pandasGEXpress import parse 

mini = parse.parse("functional_tests/mini_gctx_with_metadata_n2x3.gctx")
mini.row_metadata_df

rhd,pr_analyte_id,pr_analyte_num,pr_gene_id,pr_model_id
rid,,,,
200814_at,Analyte 11,11.0,5720.0,NaN
218597_s_at,Analyte 12,12.0,55847.0,NaN
217140_s_at,Analyte 12,12.0,7416.0,NaN


In [19]:
# now delete the column "pr_model_id" (it's all NaN anyway, so say we don't care about it)
del mini.row_metadata_df["pr_model_id"]
mini.row_metadata_df 

rhd,pr_analyte_id,pr_analyte_num,pr_gene_id
rid,,,
200814_at,Analyte 11,11.0,5720.0
218597_s_at,Analyte 12,12.0,55847.0
217140_s_at,Analyte 12,12.0,7416.0


In [20]:
# now add a column from a list 
mini.row_metadata_df["is_this_fun"] = ["yes","yes", "maybe"]

mini.row_metadata_df

rhd,pr_analyte_id,pr_analyte_num,pr_gene_id,is_this_fun
rid,,,,
200814_at,Analyte 11,11.0,5720.0,yes
218597_s_at,Analyte 12,12.0,55847.0,yes
217140_s_at,Analyte 12,12.0,7416.0,maybe


In [21]:
# BUT: This assumes the new column is ordered the same as the rids. 
# If this isn't the case:

import pandas as pd

mini = parse.parse("functional_tests/mini_gctx_with_metadata_n2x3.gctx")
mini.row_metadata_df

new_annotation = pd.DataFrame([89, 9137430, 3], index = ["217140_s_at","200814_at", "218597_s_at"], columns = ["important_metric"])
new_annotation

,important_metric
217140_s_at,89
200814_at,9137430
218597_s_at,3


In [22]:
mini.row_metadata_df = pd.merge(mini.row_metadata_df, new_annotation, left_index=True, right_index=True)
mini.row_metadata_df

,pr_analyte_id,pr_analyte_num,pr_gene_id,pr_model_id,important_metric
rid,,,,,
200814_at,Analyte 11,11.0,5720.0,NaN,9137430
218597_s_at,Analyte 12,12.0,55847.0,NaN,3
217140_s_at,Analyte 12,12.0,7416.0,NaN,89


# Transpose a GCToo

In [28]:
from broadinstitute_cmap.io.pandasGEXpress import GCToo
from broadinstitute_cmap.io.pandasGEXpress import parse 

mini = parse.parse("functional_tests/mini_gctx_with_metadata_n2x3.gctx", make_multiindex=True)
mini.multi_index_df 

bead_batch                                                                            b19  \
bead_revision                                                                          r2   
cid                                                             LJP005_A375_24H:DMSO:-666   
pr_analyte_id pr_analyte_num pr_gene_id pr_model_id rid                                     
Analyte 11    11.0           5720.0     NaN         200814_at                   -0.283359   
Analyte 12    12.0           55847.0    NaN         218597_s_at                  0.304119   
                             7416.0     NaN         217140_s_at                  0.398655   

bead_batch                                                                                        
bead_revision                                                                                     
cid                                                             LJP005_A375_24H:BRD-K76908866:10  
pr_analyte_id pr_analyte_num pr_gene_id pr_model_id rid                                           
Analyte 11    11.0           5720.0     NaN         200814_at                           0.011270  
Analyte 12    12.0           55847.0    NaN         218597_s_at                         1.921061  
                             7416.0     NaN         217140_s_at                        -0.144652

In [29]:
transposed_mini = GCToo.GCToo(data_df = mini.data_df.transpose(), 
	row_metadata_df = mini.col_metadata_df, col_metadata_df = mini.row_metadata_df, make_multiindex=True)

transposed_mini.multi_index_df

pr_analyte_id                                             Analyte 11  \
pr_analyte_num                                                  11.0   
pr_gene_id                                                   5720.0    
pr_model_id                                                      NaN   
cid                                                        200814_at   
bead_batch bead_revision rid                                           
b19        r2            LJP005_A375_24H:DMSO:-666         -0.283359   
                         LJP005_A375_24H:BRD-K76908866:10   0.011270   

pr_analyte_id                                              Analyte 12  \
pr_analyte_num                                                   12.0   
pr_gene_id                                                    55847.0   
pr_model_id                                                       NaN   
cid                                                       218597_s_at   
bead_batch bead_revision rid                                            
b19        r2            LJP005_A375_24H:DMSO:-666           0.304119   
                         LJP005_A375_24H:BRD-K76908866:10    1.921061   

pr_analyte_id                                                          
pr_analyte_num                                                         
pr_gene_id                                                    7416.0   
pr_model_id                                                       NaN  
cid                                                       217140_s_at  
bead_batch bead_revision rid                                           
b19        r2            LJP005_A375_24H:DMSO:-666           0.398655  
                         LJP005_A375_24H:BRD-K76908866:10   -0.144652

# Column or row-wise basic math

## Column/row-wise means

In [33]:
from broadinstitute_cmap.io.pandasGEXpress import parse 
import pandas as pd

mini = parse.parse("functional_tests/mini_gctx_with_metadata_n2x3.gctx", make_multiindex=True)
# by row
rowwise_mean = mini.data_df.mean(axis = 0)
# by col
colwise_mean = mini.data_df.mean(axis = 1)

# want to write to a GCToo? Easy!
colwise_mean_mini_gctoo = GCToo.GCToo(data_df = pd.DataFrame(colwise_mean, columns=["cid"]), 
	row_metadata_df = mini.row_metadata_df, col_metadata_df = pd.DataFrame(["colwise_mean"], index=["cid"]))
colwise_mean_mini_gctoo.data_df

,cid
rid,
200814_at,-0.136045
218597_s_at,1.112590
217140_s_at,0.127002


## Pairwise correlation of columns

Notes: 
- To do correlation by rows using pandas method, transpose data_df first.
- pandas' default correlation metric is pearson.

In [31]:
mini.data_df.corr(method = "spearman")

cid,LJP005_A375_24H:DMSO:-666,LJP005_A375_24H:BRD-K76908866:10
cid,,
LJP005_A375_24H:DMSO:-666,1.0,-0.5
LJP005_A375_24H:BRD-K76908866:10,-0.5,1.0


## Aggregation by specified grouping 

This tasks is particularly well-suited to the multi-index DataFrame attribute of a GCToo instance. Note that the default is NOT to create a multi-index when creating a GCToo object, so you have to request this explicitly with the `make_multiindex` argument.

In [34]:
mini.multi_index_df

bead_batch                                                                            b19  \
bead_revision                                                                          r2   
cid                                                             LJP005_A375_24H:DMSO:-666   
pr_analyte_id pr_analyte_num pr_gene_id pr_model_id rid                                     
Analyte 11    11.0           5720.0     NaN         200814_at                   -0.283359   
Analyte 12    12.0           55847.0    NaN         218597_s_at                  0.304119   
                             7416.0     NaN         217140_s_at                  0.398655   

bead_batch                                                                                        
bead_revision                                                                                     
cid                                                             LJP005_A375_24H:BRD-K76908866:10  
pr_analyte_id pr_analyte_num pr_gene_id pr_model_id rid                                           
Analyte 11    11.0           5720.0     NaN         200814_at                           0.011270  
Analyte 12    12.0           55847.0    NaN         218597_s_at                         1.921061  
                             7416.0     NaN         217140_s_at                        -0.144652

In [35]:
# Say you want to group by pr_analyte_id and pr_gene_id,
# And sum the values of these group members
mini.multi_index_df.groupby(level=["pr_analyte_id", "pr_gene_id"]).sum()

bead_batch                                     b19  \
bead_revision                                   r2   
cid                      LJP005_A375_24H:DMSO:-666   
pr_analyte_id pr_gene_id                             
Analyte 11    5720.0                     -0.283359   
Analyte 12    7416.0                      0.398655   
              55847.0                     0.304119   

bead_batch                                                 
bead_revision                                              
cid                      LJP005_A375_24H:BRD-K76908866:10  
pr_analyte_id pr_gene_id                                   
Analyte 11    5720.0                             0.011270  
Analyte 12    7416.0                            -0.144652  
              55847.0                            1.921061

# Write a GCToo instance to file 

Assume you're in an active python session in your local GCToo directory and have a GCToo instance (let's call it my_gctoo that you'd like to write to .gct or .gctx.

To write to a GCT file:

```
import write_gct 

write_gct.write(my_gctoo, "some/path/to/my_gctoo.gct") 
```

To write to a GCTX file:

```
import write_gctx

write_gctx.write(my_gctoo, "some/path/to/my_gctoo.gctx")  
```

# Convert a gct -> gctx (or vice versa) from the command line

Converting from a gct to a gctx might be useful if you have a large gct and want faster IO in the future. 

To write some_thing.gct -> some_thing.gctx in working directory:

```
python gct2gctx.py -filename some_thing.gct 
```

To write some_thing.gct to a .gctx named something_else.gctx in a different out directory (both -outname and -outpath are optional):
```
python gct2gctx.py -filename some_thing.gct -outname something_else -outpath my/special/folder
```

Converting a gctx to a gct might be useful if you want to look at your .gctx file in a text editor or something similar. 

To write some_thing.gctx -> some_thing.gct in working directory:
```
python gctx2gct.py -filename some_thing.gctx
```

To write some_thing.gctx to a .gct named something_else.gct in a different out directory (both -outname and -outpath are optional):
```
python gctx2gct.py -filename some_thing.gctx -outname something_else -outpath my/special/folder
```